# Fast Fourier Analysis
stough 202-

See [scipy's intro](https://docs.scipy.org/doc/scipy/tutorial/fft.html) as well.

When we looked at block transforms for compression (see [`view_basis_blocks`](../BlockTransform/view_basis_blocks.ipynb)), we saw how an image block could be decomposed into a combination of basic patterns. One of those pattern sets was the Discrete Cosine Transform. This happens to be very closely related to the frequency analysis we're covering here.


The [Discrete Fourier Transform](https://en.wikipedia.org/wiki/Discrete_Fourier_transform) $y[k]$ of the length-$N$ sequence $x[n]$ and its inverse are defined: 

\begin{equation}
y[k] = \sum_{n=0}^{N-1} e^{-2\pi j \frac{kn}{N}} x[n],\\
x[n] = \frac{1}{N}\sum_{k=0}^{N-1} e^{2\pi j \frac{kn}{N}} y[k]
\end{equation}

These equations show basically that any sequence $x[n]$ can be written as the sum of complex sinusoids. We will use the 2D extensions of this process.

## Imports
We'll import [scipy's `fft2` and `ifft2` functions](https://docs.scipy.org/doc/scipy/tutorial/fft.html) to help us out here.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage.color import rgb2gray
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

from scipy.fft import fft2, ifft2, fftshift, ifftshift
from skimage.transform import resize

## Image Denoising 
We can use the Fourier decomposition of an image to downweight or eliminate noise in an image. In this case the noise is high-frequency patterns.

Following along to [this demo](http://scipy-lectures.org/intro/scipy/auto_examples/solutions/plot_fft_image_denoise.html).

In [2]:
I = plt.imread('../dip_pics/moonlanding.png')
arr_info(I)

((474, 630), dtype('float32'), 0.0, 1.0)

In [3]:
plt.figure(figsize=(4,3))
plt.imshow(I, cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
im_fft = fft2(I)

In [5]:
arr_info(im_fft)

((474, 630), dtype('complex64'), (-9421.1-5242.114j), (126598.46-0j))

We see that the `fft2` of the image is an array of complex numbers, representing coefficients of certain frequency patterns in the iamge. Let's look at the magnitude spectrum. Given the vast range of values, we'll plot in the log space for visualization. 

In [6]:
plt.figure(figsize=(4,3))
plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=5), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Also, given the symmetries of the different frequency components, we can use [`fftshift`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fftshift.html) to bring the lower-frequency components to the center of the figure. This is useful because often the lower-frequency components have most of the energy or power of the signal. (Recall that the average was often a pretty good approximation to a block. This is equivalent to the zero frequency, right?)

In [7]:
plt.figure(figsize=(4,3))
plt.imshow(np.abs(fftshift(im_fft)), norm=LogNorm(vmin=5), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now let's zero-out the high-frequency components that we don't want to keep.

In [8]:
# For convenience
def plot_spectrum(im_fft, shift=True):
    plt.figure(figsize=(4,3))
    if shift:
        plt.imshow(np.abs(fftshift(im_fft)), norm=LogNorm(vmin=5), cmap='gray')
    else:
        plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=5), cmap='gray')
    plt.colorbar()

In [9]:
keep_fraction = 0.1

im_fft2 = im_fft.copy()
r, c = im_fft2.shape

im_fft2[int(r*keep_fraction):int(r*(1-keep_fraction))] = 0
im_fft2[:, int(c*keep_fraction):int(c*(1-keep_fraction))] = 0

plot_spectrum(im_fft2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There appears to be some display error. Be assured we have zero-d out some 80\% of the pixels/frequencies.

In [10]:
I_denoised = ifft2(im_fft2).real # ifft: inverse ...
arr_info(I_denoised)

((474, 630), dtype('float32'), 0.18438978, 0.79105043)

In [11]:
vis_pair(I, I_denoised, cmap='gray', second_title='Denoised', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

&nbsp;

## Let's look at the Magnitude and Phase of the FFT

The magnitude image is the magnitude of each complex number, and can be computed with `abs`, as seen above in `plot_spectrum`.

In [12]:
J = rgb2gray(plt.imread('../dip_pics/cat_small.png'))
J = resize(J, (474, 630))
arr_info(J)

/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  """Entry point for launching an IPython kernel.


((474, 630), dtype('float32'), 0.000347886, 0.97255975)

In [13]:
plt.figure(figsize=(4,3))
plt.imshow(J, cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
im_fft_J = fft2(J)
plot_spectrum(im_fft_J)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can also view the *phase* of the transform. That is we can view the angles of the complex coefficients.

In [15]:
# Thanks https://github.com/numpy/numpy/issues/3621
def phase(z):
    return z / np.abs(z)

In [16]:
plt.figure(figsize=(4,3))
plt.imshow(np.angle(fftshift(im_fft_J)), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Let's exchange the phase and magnitude of the two images.
[Cool example](https://stackoverflow.com/questions/52312053/how-to-combine-the-phase-of-one-image-and-magnitude-of-different-image-into-1-im/52337407)

In [17]:
f, ax = plt.subplots(1, 3, figsize=(8,3))
ax[0].imshow(I, cmap='gray')
ax[1].imshow(np.abs(fftshift(im_fft)), norm=LogNorm(vmin=5), cmap='gray')
ax[2].imshow(np.angle(fftshift(im_fft)), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
f, ax = plt.subplots(1, 3, figsize=(8,3))
ax[0].imshow(J, cmap='gray')
ax[1].imshow(np.abs(fftshift(im_fft_J)), norm=LogNorm(vmin=5), cmap='gray')
ax[2].imshow(np.angle(fftshift(im_fft_J)), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now let's swap the phase of J and magnitude of I, or vice versa.

In [19]:
combined = np.multiply(np.abs(im_fft), np.exp(1j*np.angle(im_fft_J)))
imgCombined = np.real(np.fft.ifft2(combined))
plt.figure(figsize=(4,3))
plt.imshow(np.clip(imgCombined, 0,1), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
combined = np.multiply(np.abs(im_fft_J), np.exp(1j*np.angle(im_fft)))
imgCombined = np.real(np.fft.ifft2(combined))
plt.figure(figsize=(4,3))
plt.imshow(np.clip(imgCombined, 0,1), cmap='gray');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …